### Overview: 
- This script identifies stocks exhibiting sustained relative strength against market indices. 
- It specifically filters for equities breaking out above the upper 50-day Bollinger Band (2σ limit).<br>
<b>Trading Rules <br></b>
<b>Entry (Buy):</b>
- Stock must maintain higher weekly strength than the index for 4 consecutive weeks.
- Closing price must exceed the upper Bollinger Band (50-period MA, 2 standard deviations).<br>
<b>Exit (Sell):</b>
- Close below the 100-day Moving Average (MA).
- Price drops below a trailing stop defined as $2 \times ATR(13)$.

In [1]:
import pandas as pa
from GetFreshMarketData import *

In [2]:
from datetime import datetime, timedelta

# Get current date and time
now = datetime.now()
now.weekday()

2

In [2]:
end_date = datetime.today()
start_date = end_date - timedelta(days = 365 * 3)
past_six_months = end_date - timedelta(days = 90)
file_cr = 50000000
weekly_relative_performace = 4
buy_bb_band_period = 50
buy_bb_band_sigma = 2
sell_sma = 100
sell_atr_period = 13
sell_atr_mult = 2

all_traded_stock = pd.read_csv(INDEX_DIR_CONST/'all_traded_stock.csv',low_memory=False)
all_traded_stock.date = all_traded_stock.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
all_traded_stock = all_traded_stock.loc[((all_traded_stock.date <= end_date) & 
                                        (all_traded_stock.date >= past_six_months) &
                                        (all_traded_stock.total_value >=file_cr)),:]
all_traded_stock = all_traded_stock.groupby('symbol').agg({'date':len}).reset_index()
candidate_symbol = all_traded_stock.loc[all_traded_stock.date==all_traded_stock.date.max(),'symbol']

In [3]:
progress_bar = tqdm(INDEX_TICKERS_DF.iterrows(), desc="Downloading historical data", total=len(INDEX_TICKERS_DF.NSE_SYMBOL))

relative_performace_df = pd.DataFrame(columns =['index_name','roc'])
index_contituents_df = pd.DataFrame(columns = ['symbol','index_name'])
weekly_index_roc = {}
all_index_dict = {}
for _, row in progress_bar:
    symbol = row['NSE_SYMBOL']
    fl_name = row['INDEX_NAME']
    index  = pd.read_csv(STOCK_DIR/f'{symbol}.csv')
    index_constituents = pd.read_csv(INDEX_DIR_CONST/f'{fl_name}.csv')
    index.date = pd.to_datetime(index.date)
    index = index.loc[index.date <= end_date,:]
    index = index.set_index('date')
    
    index = index.sort_index()
    
    index_constituents['index_name'] = symbol
    index_contituents_df = pd.concat([index_contituents_df,index_constituents[['symbol','index_name']]])
    
    weekly_index_close = index.resample('W-MON').agg({'close': 'last'})
    weekly_index_close = weekly_index_close.sort_index()
    weekly_index_close = weekly_index_close.ffill()
    weekly_performace_close = weekly_index_close.pct_change(1)
    

    
    if relative_performace_df.empty:
        relative_performace_df = pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_performace_close.tail(1).values[0]})
    else:
        relative_performace_df = pd.concat([relative_performace_df,pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_performace_close.tail(1).values[0]})])
    
    weekly_index_roc[symbol] = weekly_performace_close
    all_index_dict[symbol] = index


relative_performace_df = relative_performace_df.sort_values(by='roc')

In [4]:
def calculate_bollinger_bands(close: pd.Series, period: int = buy_bb_band_period, sigma: int = buy_bb_band_sigma) -> pd.DataFrame:
                """Calculate Bollinger Bands"""
                rolling_mean = close.rolling(window=period).mean()
                rolling_std = close.rolling(window=period).std()
                upper_band = rolling_mean + (rolling_std * sigma)
                lower_band = rolling_mean - (rolling_std * sigma)
                return pd.DataFrame({
                    'upper_band': upper_band,
                    'lower_band': lower_band
                }, index=close.index)

In [5]:
pbar = tqdm(candidate_symbol,desc = "Processing for ",total=len(candidate_symbol))
final_stocks = pd.DataFrame()
for symbol in pbar:
    pbar.set_description(f'Processing for {symbol}')
    file = STOCK_DIR/f'{symbol}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.loc[df.date <= end_date,:]
        df = df.set_index('date')
        df = df.sort_index()

        weekly_stock_close = df.resample('W-MON').agg({'close': 'last'})
        weekly_stock_close = weekly_stock_close.sort_index()
        weekly_stock_close = weekly_stock_close.ffill()
        weekly_stock_performance = weekly_stock_close.pct_change(1)

        relative_performace_count = 0
        indexes_beaten = []
        for index_name,index_weekly_performace in weekly_index_roc.items():
            relative_performace = (weekly_stock_performance.tail(weekly_relative_performace) > index_weekly_performace.tail(weekly_relative_performace)).sum()
            if relative_performace.values[0] >= weekly_relative_performace:
                relative_performace_count +=1
                indexes_beaten.append(index_name)

        
        # if relative_performace_count >= 6:
        #     print(symbol, relative_performace_count,indexes_beaten)
            
        
        if df['close'].shape[0]>50 and weekly_stock_performance.tail(1).values[0] >= relative_performace_df.roc.iloc[-1] and relative_performace_count >= 6:
            bb = calculate_bollinger_bands(df['close'])
            latest_close = df['close'].iloc[-1]
            latest_upper = bb['upper_band'].iloc[-1]
            if pd.notna(latest_upper) and latest_close > latest_upper:
                temp = pd.DataFrame({'symbol': symbol,
                                    'weekly_return': weekly_stock_performance.tail(1).values[0],
                                    'latest_close': latest_close,
                                    'upper_bb': latest_upper,
                                    'bb_breakout': latest_close / latest_upper - 1,
                                    'indexes_beaten':','.join(indexes_beaten),
                                    'indexes_beaten_count': relative_performace_count
                                    })
                final_stocks = pd.concat([final_stocks, temp])
if not final_stocks.empty:
    final_stocks = final_stocks.sort_values(by='weekly_return', ascending=False)
    

Processing for ZYDUSLIFE: 100%|██████████| 440/440 [00:50<00:00,  8.70it/s] 


In [7]:
final_stocks.to_csv(TEMP/'final_stock.csv')
final_stocks.head(20)

,symbol,weekly_return,latest_close,upper_bb,bb_breakout,indexes_beaten,indexes_beaten_count
0,CUPID,0.093552,447.70,441.886940,0.013155,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,SILVERETF,0.043800,194.70,192.258338,0.012700,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,SILVERIETF,0.042728,201.09,197.389199,0.018749,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,TATSILV,0.042644,19.56,19.329751,0.011912,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,SBISILVER,0.042515,197.64,194.556298,0.015850,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",21
0,SILVERBEES,0.041597,192.56,189.925210,0.013873,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,SILVER,0.041481,200.86,198.222703,0.013305,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,HDFCSILVER,0.041359,193.12,191.525496,0.008325,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",21
0,ASHOKLEY,0.040403,173.56,171.135788,0.014165,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",20
0,HINDZINC,0.036440,588.75,563.423269,0.044952,"NIFTY MIDCAP 100,NIFTY SMLCAP 100,NIFTY MEDIA,...",11


In [9]:
# check for stock which are giving sell signal
stocks_holding = ['EICHERMOT','MUTHOOTFIN','BAJFINANCE','HINDZINC','BHARTIARTL','BHARTIARTL','BHARTIARTL']
pbar = tqdm(stocks_holding, desc = "Processing for",total=len(stocks_holding))

sell_signal_stocks = pd.DataFrame()
for symbol in pbar:
    pbar.set_description(f'Processing for {symbol}')
    file = STOCK_DIR/f'{symbol}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.set_index('date')
        df = df.sort_index()        
        if len(df) < max(sell_sma, sell_atr_period + 1):
            raise Exception(f'historical less than {max(sell_sma, sell_atr_period + 1)} for {symbol}')
        df['SMA100'] = df['close'].rolling(sell_sma).mean()

        high, low, close = df['high'], df['low'], df['close']
        prev_close = close.shift(1)
        tr1 = high - low
        tr2 = abs(high - prev_close)
        tr3 = abs(low - prev_close)
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df['ATR'] = tr.rolling(sell_atr_period).mean()
        # Latest conditions
        latest, prev = df.iloc[-1], df.iloc[-2]
        below_sma = latest['close'] < latest['SMA100']
        big_drop = latest['close'] < (prev['close'] - sell_atr_mult * latest['ATR'])
        if below_sma or big_drop:
            if sell_signal_stocks.empty:
                sell_signal_stocks = pd.DataFrame({'symbol':[symbol],'close':latest['close'],'below_sma':[below_sma],'big_drop':[big_drop]})
            else:
                sell_signal_stocks = pd.concat([sell_signal_stocks, 
                                                pd.DataFrame({'symbol':[symbol],'close':latest['close'],'below_sma':[below_sma],'big_drop':[big_drop]})])
    else:
        raise Exception(f'historical data not found for {symbol}')
    


Processing for BHARTIARTL: 100%|██████████| 7/7 [00:00<00:00, 10.60it/s]


In [10]:
sell_signal_stocks

""
